# Project 550-01-Dtree
## Team members
### 1) Sean Pereira - Sean.Pereira@student.csulb.edu
### 2) Sushmitha Pasala - Sushmitha.Pasala@student.csulb.edu
### 3) Vatsal Patel - Vatsal.Patel01@student.csulb.edu




# THE CODE
## Created a Class containing all the required datatypes, and functions for building Decision Tree.
<ol>
    <li>DATA TYPES
        <ul>
            <li>df - DataFrame to save csv file content and to perform easy panda functions</li>
            <li>left - left subtree/classifier</li>
            <li>right - right subtree/classifier</li>
            <li>fkey - feature name on which it decides classification</li>
            <li>fval - Feature value which is used to divide data set for classifing</li>
            <li>depth - gives current tree depth</li>
            <li>max_depth - gives max discoverable depth</li>
            <li>target - label to be classified for</li>
        </ul>
    </li>
    <li>FUNCTIONS
        <ul>
            <li>init - Initializes all values for the object
                <ul>
                    <li>Parameters - 
                        <ul>
                            <li>Depth - current depth of tree</li>
                            <li>Max_depth - Maximum depth</li>
                        </ul>
                    </li>
                    <li>Returns - None</li>
                </ul>
            </li>
            <li>label_output - Alters label for convinency of code
                <ul>
                    <li>Parameters - None</li>
                    <li>Returns - None</li>
                </ul>
            </li>
            <li>processing_data - Converts multiple values to binary values and generating n number of columns from 1 column
                <ul>
                    <li>Parameters - 
                        <ul>
                            <li>Data - the DataFrame</li>
                        </ul>
                    </li>
                    <li>Returns - 
                        <ul>
                            <li>5 Dfs - 5 different dataframes for each feature vector</li>
                        </ul>
                    </li>
                </ul>
            </li>
            <li>entropy - calculates entropy of desired feature 
                <ul>
                    <li>Parameters - 
                        <ul>
                            <li>Col - columns of the record of which entropy has to be calculated</li>
                        </ul>
                    </li>
                    <li>Returns - entropy as float</li>
                </ul>
            </li>
            <li>information_gain - Calculates information gain from entropy of possible nodes 
                <ul>
                    <li>Parameters - 
                        <ul>
                            <li>x_data - Feature Data set</li>
                            <li>fkey - Feature name</li>
                            <li>fval - Feature Value</li>
                        </ul>
                    </li>
                    <li>Returns - 
                        <ul>
                            <li>info_gain - returns information gain</li>
                        </ul>
                    </li>
                </ul>
            </li>
            <li>divide_data - Divides the data set for classifying using maximum all info gains
                <ul>
                    <li>Parameters - 
                        <ul>
                            <li>x_data - Feature Data set</li>
                            <li>fkey - Feature name</li>
                            <li>fval - Feature Value</li>
                        </ul>
                    </li>
                    <li>Returns - 
                        <ul>
                            <li>X_left - left split Dataset</li>
                            <li>X_right - right split Dataset</li>
                        </ul>
                    </li>
                </ul>
            </li>
            <li>frequency_of_Output - gives frequency of the labels present in x_train datset
                <ul>
                    <li>Parameters - 
                        <ul>
                            <li>X_train - training Data set</li>
                        </ul>
                    </li>
                    <li>Returns - 
                        <ul>
                            <li>Max(frequent label)</li>
                        </ul>
                    </li>
                </ul>
            </li>
            <li>train - calls different functions and trains the decision tree using training dataset ans its labels
                <ul>
                    <li>Parameters - 
                        <ul>
                            <li>X_train - training Data set</li>
                        </ul>
                    </li>
                    <li>Returns - None
                    </li>
                </ul>
            </li>
            <li>predict - label to be classified for
                <ul>
                    <li>Parameters - 
                        <ul>
                            <li>Test - Testing Data set</li>
                        </ul>
                    </li>
                    <li>Returns - 
                        <ul>
                            <li>Predicted class</li>
                        </ul>
                    </li>
                </ul>
            </li>
            <li>dataframe - label to be classified for
                <ul>
                    <li>Parameters - None
                    </li>
                    <li>Returns - 
                        <ul>
                            <li>whole Dataframe</li>
                        </ul>
                    </li>
                </ul>
            </li>
        </ul>
    </li>
</ol>

### INSTALLS Numpy and Pandas library to run the rest of the code

In [1]:
import subprocess

command = "pip install numpy" #command to be executed 
res = subprocess.call(command, shell = True)

command = "pip install pandas" #command to be executed
res = subprocess.call(command, shell = True)

##### Code starts here

In [2]:
import pandas as pd
import numpy as np
import random
from pprint import pprint

## Class DecisionTree

In [3]:
class DecisionTree:
    def __init__(self,depth=0,max_depth=5):
        #Read the data from csv file and name the columns
        
        c=['White King file (column)','White King rank (row)','White Rook file','White Rook rank','Black King file','Black King rank','Output']
        self.df=pd.read_csv('550-p1-cset-krk-1.csv',header=None)
        self.df=self.df.rename({0:'White King file (column)',1:'White King rank (row)',2:'White Rook file',3:'White Rook rank',4:'Black King file',5:'Black King rank',6:'Output'}, axis=1)
        df0,df1,df2,df3,df4,df5=self.processing_data(self.df)
        self.label_output()
        self.df=pd.concat([df0,df1,df2,df3,df4,df5,self.df['Output']],axis=1)
        self.left=None # left branch
        self.right=None # right branch
        self.fkey=None 
        self.fval=None
        self.depth=depth
        self.max_depth=max_depth
        self.target=None
        self.d1={17:'draw',0:'zero',1:'one',2:'two',3:'three',4:'four',5:'five',6:'six',7:'seven',8:'eight',9:'nine',10:'ten',11:'eleven',12:'twelve',13:'thirteen',14:'fourteen',15:'fifteen',16:'sixteen'}
        
    def label_output(self):
        #Converts Labels to int numbers
        self.d={'draw':17,'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9,'ten':10,'eleven':11
          ,'twelve':12,'thirteen':13,'fourteen':14,'fifteen':15,'sixteen':16}
        #iterating over each column in dataframe
        for column in self.df:
            if column=='Output':
                s1=self.df[column].values
                for j,i in enumerate(s1):
                    s1[j]=self.d[i]
                break
        self.df=self.df.assign(Output=s1,inplace='True')

        
        
    def processing_data(self,data):
        # Labeling each data to 0-1, converting categorical to numerical data
        
        columns_text_0=['White King file (a)','White King file (b)','White King file (c)','White King file (d)','White King rank (e)','White King file (f)','White King rank (g)','White King file (h)']
        columns_data_0=['White King rank (1)','White King rank (2)','White King rank (3)','White King rank (4)','White King rank (5)','White King rank (6)','White King rank (7)','White King rank (8)']
        columns_text_1=['White Rook file (a)','White Rook file (b)','White Rook file (c)','White Rook file (d)','White Rook file (e)','White Rook file (f)','White Rook file (g)','White Rook file (h)']
        columns_data_1=['White Rook rank (1)','White Rook rank (2)','White Rook rank (3)','White Rook rank (4)','White Rook rank (5)','White Rook rank (6)','White Rook rank (7)','White Rook rank (8)']
        columns_text_2=['Black King file (a)','Black King file (b)','Black King file (c)','Black King file (d)','Black King file (e)','Black King file (f)','Black King file (g)','Black King file (h)']
        columns_data_2=['Black King rank (1)','Black King rank (2)','Black King rank (3)','Black King rank (4)','Black King rank (5)','Black King rank (6)','Black King rank (7)','Black King rank (8)']
        index=0
#         grabing each column to change it with binary columns
        for i in ['White King file (column)','White King rank (row)','White Rook file','White Rook rank','Black King file','Black King rank']:
            alphabets=[]
            numericals=[]
            for columndata in data[i]:
                letter=[0]*8
                numbers=[0]*8
                if not isinstance(columndata, int):
                    letter[ord(columndata)-ord('a')]=1
                    alphabets.append(letter)
                else:
                    numbers[ord(str(columndata))-ord('0')-1]=1
                    numericals.append(numbers)
#             Updating dataframe for the data manupilation
            if index==0:
                df0=pd.DataFrame(data=alphabets, columns=columns_text_0)
            if index==1:
                df1=pd.DataFrame(data=numericals, columns=columns_data_0)
            if index==2:
                df2=pd.DataFrame(data=alphabets, columns=columns_text_1)
            if index==3:
                df3=pd.DataFrame(data=numericals, columns=columns_data_1)
            if index==4:
                df4=pd.DataFrame(data=alphabets, columns=columns_text_2)
            if index==5:
                df5=pd.DataFrame(data=numericals, columns=columns_data_2)
            index+=1
        return (df0,df1,df2,df3,df4,df5)
# returns each column's dataframes
    
    def entropy(self,col):
        #calculates Entropy using log formula
        counts=np.unique(col,return_counts=True)
        ent=0.0
        for i in counts[1]:
#             probability
            p=i/col.shape[0]
#             calculating entropy
            ent+=(-1.0*p*np.log2(p))
            #heres the formula
        return ent
    #returns entropy
    
    def information_gain(self,x_data,fkey,fval):# calculates information gain from entropy of possible nodes
        right,left=self.divide_data(x_data,fkey,fval)
        l=float(left.shape[0])/x_data.shape[0]
        r=float(right.shape[0])/x_data.shape[0]
        if left.shape[0]==0 or right.shape[0]==0:
            return float("-inf")
        print("Average entropy:",(l*self.entropy(left.Output)+r*self.entropy(right.Output)))
#         finding information gain 
        i_gain=self.entropy(x_data.Output)-(l*self.entropy(left.Output)+r*self.entropy(right.Output))
        return i_gain#returns information gain of that probable node
    
    def divide_data(self,x_data,fkey,fval):
        #generates two dataframe as per the feature and its dividing value
        #fkey: Feature names 
        #fval: Feature values
        
        x_right=pd.DataFrame([],columns=x_data.columns)
        x_left=pd.DataFrame([],columns=x_data.columns)
        for i in range(x_data.shape[0]):
            val = x_data[fkey].loc[i]
            if val >= fval:
                x_right = x_right.append(x_data.iloc[i])
            else:
                x_left = x_left.append(x_data.iloc[i])
        return x_right,x_left#returns left and right dataframes
    
    def frequency_of_Output(self, x_train):
        
        self.dict={}
        for i in x_train:
            if i not in self.dict:
                self.dict[i]=1
            else:
                self.dict[i]+=1
        return max(self.dict, key= lambda d: self.dict[d])
        
    def train(self,x_train): #calls different functions and trains the decision tree using training dataset ans its labels
        features=self.df.columns[:-1]
        info_gains=[]
        for i in features:
            print("i = ",i)
            i_gain=self.information_gain(x_train,i,0.5)
            info_gains.append(i_gain)
            print("Information gain:", info_gains[-1])
        #print("INFO GAINS : ",info_gains)
        self.fkey=features[np.argmax(info_gains)]
        self.fval=0.5
        print("Splitting Tree ",self.fkey," with info gain ",max(info_gains))
        data_right,data_left=self.divide_data(x_train,self.fkey,self.fval)
        data_right=data_right.reset_index(drop=True)
        data_left=data_left.reset_index(drop=True)
        if data_left.shape[0]==0 or data_right.shape[0]==0:
            
            self.target=self.d1[self.frequency_of_Output(x_train.Output)]
            return 
        if self.depth>=self.max_depth:
            
            self.target=self.d1[self.frequency_of_Output(x_train.Output)]
            return 
        self.left=DecisionTree(self.depth+1,self.max_depth)
        self.left.train(data_left)
        self.right=DecisionTree(self.depth+1,self.max_depth)
        self.right.train(data_right)

        self.target=self.d1[self.frequency_of_Output(x_train.Output)]
        return 
    
    def predict(self,test): #predicts the possible classification
        if test[self.fkey] > self.fval:
            if self.right is None:
                return self.target
            return self.right.predict(test)
        if test[self.fkey] <= self.fval:
            if self.left is None:
                return self.target
            return self.left.predict(test)
    def dataframe(self):# returns whole dataframe
        return self.df

    

        
#Creating Object of first Decision Tree
d=DecisionTree()



# Splitting Data Into training, test and validate :60,20,20
train_data, validate_data, test_data = np.split(d.dataframe().sample(frac=1,random_state=42), [int(.6*len(d.dataframe())), int(.8*len(d.dataframe()))])

#Reset Index to 0
train_data=train_data.reset_index(drop=True)
test_data=test_data.reset_index(drop=True)

# Building tree
d.train(train_data)

i =  White King file (a)
Average entropy: 3.3953568632487197
Information gain: 0.03811799150227202
i =  White King file (b)
Average entropy: 3.3381945294413846
Information gain: 0.09528032530960706
i =  White King file (c)
Average entropy: 3.166781144099489
Information gain: 0.2666937106515026
i =  White King file (d)
Average entropy: 3.153549127626326
Information gain: 0.2799257271246658
i =  White King rank (e)
Information gain: -inf
i =  White King file (f)
Information gain: -inf
i =  White King rank (g)
Information gain: -inf
i =  White King file (h)
Information gain: -inf
i =  White King rank (1)
Average entropy: 3.345126196103379
Information gain: 0.08834865864761277
i =  White King rank (2)
Average entropy: 3.3791349921427907
Information gain: 0.054339862608201006
i =  White King rank (3)
Average entropy: 3.334716885134373
Information gain: 0.09875796961661854
i =  White King rank (4)
Average entropy: 3.300884786792241
Information gain: 0.1325900679587506
i =  White King rank (5

Average entropy: 2.926108593725078
Information gain: 0.0947092291868441
i =  Black King rank (7)
Average entropy: 2.944109091526044
Information gain: 0.07670873138587808
i =  Black King rank (8)
Average entropy: 2.926606404917745
Information gain: 0.0942114179941771
Splitting Tree  Black King file (a)  with info gain  0.32327386382137524
i =  White King file (a)
Average entropy: 2.4224026467103648
Information gain: 0.045183118200161854
i =  White King file (b)
Average entropy: 2.3006999477730465
Information gain: 0.16688581713748007
i =  White King file (c)
Average entropy: 2.2835573101988
Information gain: 0.18402845471172657
i =  White King file (d)
Average entropy: 2.2696276917420692
Information gain: 0.19795807316845737
i =  White King rank (e)
Information gain: -inf
i =  White King file (f)
Information gain: -inf
i =  White King rank (g)
Information gain: -inf
i =  White King file (h)
Information gain: -inf
i =  White King rank (1)
Average entropy: 2.278419099619562
Information ga

Average entropy: 2.0149410670028725
Information gain: 0.08024909876817432
i =  Black King rank (6)
Average entropy: 1.9505474945224284
Information gain: 0.1446426712486184
i =  Black King rank (7)
Average entropy: 2.00169443339965
Information gain: 0.09349573237139674
i =  Black King rank (8)
Average entropy: 1.9950285941626518
Information gain: 0.100161571608395
Splitting Tree  Black King rank (2)  with info gain  0.396641067962866
i =  White King file (a)
Average entropy: 1.285266182301252
Information gain: 0.04416782062371327
i =  White King file (b)
Average entropy: 1.1840285548190022
Information gain: 0.145405448105963
i =  White King file (c)
Average entropy: 1.1415414066556502
Information gain: 0.18789259626931498
i =  White King file (d)
Average entropy: 1.1084585933443496
Information gain: 0.22097540958061557
i =  White King rank (e)
Information gain: -inf
i =  White King file (f)
Information gain: -inf
i =  White King rank (g)
Information gain: -inf
i =  White King file (h)
I

Information gain: -inf
i =  White Rook file (e)
Average entropy: 1.36207461902997
Information gain: 0.19920350542916299
i =  White Rook file (f)
Information gain: -inf
i =  White Rook file (g)
Average entropy: 1.3443609377704335
Information gain: 0.21691718668869941
i =  White Rook file (h)
Average entropy: 1.2677136812595367
Information gain: 0.2935644431995963
i =  White Rook rank (1)
Information gain: -inf
i =  White Rook rank (2)
Information gain: -inf
i =  White Rook rank (3)
Average entropy: 1.2012050593046015
Information gain: 0.3600730651545314
i =  White Rook rank (4)
Average entropy: 1.2677136812595367
Information gain: 0.2935644431995963
i =  White Rook rank (5)
Average entropy: 1.0943609377704333
Information gain: 0.46691718668869964
i =  White Rook rank (6)
Average entropy: 1.438721875540867
Information gain: 0.12255624891826589
i =  White Rook rank (7)
Information gain: -inf
i =  White Rook rank (8)
Information gain: -inf
i =  Black King file (a)
Information gain: -inf
i 

Average entropy: 0.9182958340544896
Information gain: 0.07278022578373267
i =  Black King file (g)
Average entropy: 0.8888888888888888
Information gain: 0.10218717094933338
i =  Black King file (h)
Average entropy: 0.8999850522344305
Information gain: 0.09109100760379174
i =  Black King rank (1)
Information gain: -inf
i =  Black King rank (2)
Information gain: -inf
i =  Black King rank (3)
Information gain: -inf
i =  Black King rank (4)
Information gain: -inf
i =  Black King rank (5)
Information gain: -inf
i =  Black King rank (6)
Information gain: -inf
i =  Black King rank (7)
Information gain: -inf
i =  Black King rank (8)
Information gain: -inf
Splitting Tree  White King file (c)  with info gain  0.3788788371352292
i =  White King file (a)
Information gain: -inf
i =  White King file (b)
Average entropy: 1.3333333333333333
Information gain: 0.5849625007211563
i =  White King file (c)
Average entropy: 1.2682734124061352
Information gain: 0.6500224216483543
i =  White King file (d)
Ave

...

Information gain: -inf
i =  White King rank (5)
Information gain: -inf
i =  White King rank (6)
Information gain: -inf
i =  White King rank (7)
Information gain: -inf
i =  White King rank (8)
Information gain: -inf
i =  White Rook file (a)
Average entropy: 1.6365990399546155
Information gain: 0.5098403047163997
i =  White Rook file (b)
Information gain: -inf
i =  White Rook file (c)
Average entropy: 1.9419370270634915
Information gain: 0.2045023176075238
i =  White Rook file (d)
Average entropy: 2.0550491140495653
Information gain: 0.09139023062145002
i =  White Rook file (e)
Average entropy: 2.04903154372896
Information gain: 0.09740780094205537
i =  White Rook file (f)
Average entropy: 2.077468164834898
Information gain: 0.06897117983611745
i =  White Rook file (g)
Average entropy: 1.8600423875550591
Information gain: 0.28639695711595614
i =  White Rook file (h)
Average entropy: 1.789151616960017
Information gain: 0.35728772771099826
i =  White Rook rank (1)
Average entropy: 2.077468

Average entropy: 1.418777486967189
Information gain: 0.1955533481920766
i =  White Rook rank (8)
Average entropy: 1.572623640813343
Information gain: 0.04170719434592263
i =  Black King file (a)
Average entropy: 1.5007519234097646
Information gain: 0.11357891174950097
i =  Black King file (b)
Average entropy: 1.4880752283035696
Information gain: 0.12625560685569592
i =  Black King file (c)
Average entropy: 1.4880752283035696
Information gain: 0.12625560685569592
i =  Black King file (d)
Average entropy: 1.2230872715300098
Information gain: 0.3912435636292557
i =  Black King file (e)
Information gain: -inf
i =  Black King file (f)
Information gain: -inf
i =  Black King file (g)
Average entropy: 1.2230872715300098
Information gain: 0.3912435636292557
i =  Black King file (h)
Information gain: -inf
i =  Black King rank (1)
Information gain: -inf
i =  Black King rank (2)
Information gain: -inf
i =  Black King rank (3)
Information gain: -inf
i =  Black King rank (4)
Information gain: -inf
i

Information gain: -inf
i =  Black King file (d)
Information gain: -inf
i =  Black King file (e)
Information gain: -inf
i =  Black King file (f)
Information gain: -inf
i =  Black King file (g)
Information gain: -inf
i =  Black King file (h)
Information gain: -inf
i =  Black King rank (1)
Information gain: -inf
i =  Black King rank (2)
Information gain: -inf
i =  Black King rank (3)
Information gain: -inf
i =  Black King rank (4)
Information gain: -inf
i =  Black King rank (5)
Information gain: -inf
i =  Black King rank (6)
Information gain: -inf
i =  Black King rank (7)
Information gain: -inf
i =  Black King rank (8)
Information gain: -inf
Splitting Tree  White King file (a)  with info gain  -inf
i =  White King file (a)
Information gain: -inf
i =  White King file (b)
Information gain: -inf
i =  White King file (c)
Information gain: -inf
i =  White King file (d)
Information gain: -inf
i =  White King rank (e)
Information gain: -inf
i =  White King file (f)
Information gain: -inf
i =  Wh

Information gain: -inf
i =  Black King rank (4)
Information gain: -inf
i =  Black King rank (5)
Information gain: -inf
i =  Black King rank (6)
Information gain: -inf
i =  Black King rank (7)
Information gain: -inf
i =  Black King rank (8)
Information gain: -inf
Splitting Tree  White Rook rank (5)  with info gain  0.9182958340544896
i =  White King file (a)
Information gain: -inf
i =  White King file (b)
Information gain: -inf
i =  White King file (c)
Information gain: -inf
i =  White King file (d)
Information gain: -inf
i =  White King rank (e)
Information gain: -inf
i =  White King file (f)
Information gain: -inf
i =  White King rank (g)
Information gain: -inf
i =  White King file (h)
Information gain: -inf
i =  White King rank (1)
Information gain: -inf
i =  White King rank (2)
Information gain: -inf
i =  White King rank (3)
Information gain: -inf
i =  White King rank (4)
Information gain: -inf
i =  White King rank (5)
Information gain: -inf
i =  White King rank (6)
Information gain

Information gain: -inf
i =  White King rank (1)
Information gain: -inf
i =  White King rank (2)
Information gain: -inf
i =  White King rank (3)
Information gain: -inf
i =  White King rank (4)
Information gain: -inf
i =  White King rank (5)
Information gain: -inf
i =  White King rank (6)
Information gain: -inf
i =  White King rank (7)
Information gain: -inf
i =  White King rank (8)
Information gain: -inf
i =  White Rook file (a)
Information gain: -inf
i =  White Rook file (b)
Information gain: -inf
i =  White Rook file (c)
Information gain: -inf
i =  White Rook file (d)
Average entropy: 0.0
Information gain: 0.0
i =  White Rook file (e)
Average entropy: 0.0
Information gain: 0.0
i =  White Rook file (f)
Average entropy: 0.0
Information gain: 0.0
i =  White Rook file (g)
Information gain: -inf
i =  White Rook file (h)
Information gain: -inf
i =  White Rook rank (1)
Information gain: -inf
i =  White Rook rank (2)
Information gain: -inf
i =  White Rook rank (3)
Information gain: -inf
i =  

In [4]:
def decision_tree_algorithm(d, counter=0):         
    
    if (d.left is None) and (d.right is None) :
        return d.target
    fval = random.random()
    counter += 1
    # instantiate sub-tree
    question = "{} <= {}".format(d.fkey, fval)
    sub_tree = {question: []}

    # find answers (recursion)
    yes_answer = decision_tree_algorithm(d.left, counter)
    no_answer = decision_tree_algorithm(d.right, counter)

    # If the answers are the same, then there is no point in asking the qestion.
    # This could happen when the data is classified even though it is not pure
    # yet (min_samples or max_depth base cases).
    if yes_answer == no_answer:
        sub_tree = yes_answer
    else:
        sub_tree[question].append(yes_answer)
        sub_tree[question].append(no_answer)

    return sub_tree

In [5]:
tree = decision_tree_algorithm(d)
pprint(tree)

{'Black King rank (1) <= 0.5028392798092631': [{'Black King file (a) <= 0.8177495346954151': [{'Black King file (b) <= 0.5975471225294816': [{'Black King rank (2) <= 0.7744292622211487': [{'Black King file (h) <= 0.690203717417678': ['draw',
                                                                                                                                                                                                                                          'nine']},
                                                                                                                                                                                            {'White Rook rank (3) <= 0.49344315974641706': ['nine',
                                                                                                                                                                                                                                            'draw']}]},
                      

### Train DataSet

Here we show the sneak peak of the train dataset containing 49 columns and 132 rows including output label

That was randomly selected from the whole dataset and this consists of 60% of the whole dataset.

And this consist some of the left over predictions from the previous training with probability of 3 times more than previous.

In [6]:
train_data.head()

,White King file (a),White King file (b),White King file (c),White King file (d),White King rank (e),White King file (f),White King rank (g),White King file (h),White King rank (1),White King rank (2),...,Black King file (h),Black King rank (1),Black King rank (2),Black King rank (3),Black King rank (4),Black King rank (5),Black King rank (6),Black King rank (7),Black King rank (8),Output
0,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,6
1,0,0,0,1,0,0,0,0,0,1,...,1,1,0,0,0,0,0,0,0,6
2,0,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,4
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,8
4,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,17


### Validation DataSet

Here we show the sneak peak of the validation dataset containing 49 columns and 44 rows including output label

That was randomly selected from the whole dataset and this consists of 20% of the whole dataset.

In [7]:
validate_data.head()

,White King file (a),White King file (b),White King file (c),White King file (d),White King rank (e),White King file (f),White King rank (g),White King file (h),White King rank (1),White King rank (2),...,Black King file (h),Black King rank (1),Black King rank (2),Black King rank (3),Black King rank (4),Black King rank (5),Black King rank (6),Black King rank (7),Black King rank (8),Output
170,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,8
64,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2
44,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
143,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,6
40,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1


### Holdout DataSet

Here we show the sneak peak of the test/holdout dataset containing 49 columns 44 rows including output label

That was randomly selected from the whole dataset and this consists of 20% of the whole dataset.

In [8]:
test_data.head()

,White King file (a),White King file (b),White King file (c),White King file (d),White King rank (e),White King file (f),White King rank (g),White King file (h),White King rank (1),White King rank (2),...,Black King file (h),Black King rank (1),Black King rank (2),Black King rank (3),Black King rank (4),Black King rank (5),Black King rank (6),Black King rank (7),Black King rank (8),Output
0,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,6
1,0,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,8
2,0,0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,9
3,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,2
4,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2


# BAGGING Function

<ol>
    <li>FUNCTIONS
        <ul>
            <li>bagging_replacement - Adds falsily classified entries of holdout to training with 3 time probability
                for better trial of training decision tree
                <ul>
                    <li>Parameters - 
                        <ul>
                            <li>t_set - previous training set</li>
                            <li>holdout_set - current holdout set</li>
                            <li>d - Tree Object</li>
                        </ul>
                    </li>
                    <li>Returns - 
                        <ul>
                            <li>final_t_set - updated training set</li>
                            <li>final_holdout_set - same holdout set</li>
                        </ul>
                    </li>
                </ul>
            </li>
            <li>accuracy - Checks for inaccuracy
                <ul>
                    <li>Parameters - 
                        <ul>
                            <li>test_set - Test Data set</li>
                            <li>d - Tree Object</li>
                        </ul>
                    </li>
                    <li>Returns - 
                        <ul>
                            <li>returns accuracy</li>
                        </ul>
                    </li>
                </ul>
            </li>
        </ul>
    </li>
</ol>

In [9]:
def bagging_susbstition(train_set, holdout_set,d): 
    
    # adds falsily classified entries of holdout to training with 3 time probability
    #for better trial of training decision tree

    
    Training_indexes = list(train_set.index)
    Testing_indexes = list(holdout_set.index)
    combined_set=Training_indexes
    combined_set.sort()
    
    
    final_train_set = []
    final_holdout_set = []
    
    incorrect_array=accuracy(d,holdout_set)[1]


    for i in incorrect_array:
        combined_set.append(i)
        combined_set.append(i)

    for _ in range(len(train_set)):
        index = random.randint(0, len(train_set) - 1)
        final_train_set.append(combined_set[index])


    # makining sure duplicate indixes are not in final_train_set
    for index_value in combined_set:
        if index_value not in final_holdout_set:
            final_holdout_set.append(index_value)
    
    for index_value in final_train_set:
        if index_value in final_holdout_set:
            final_holdout_set.remove(index_value)
        
            
    if len(final_holdout_set)> len(Testing_indexes):
        final_holdout_set=final_holdout_set[:44]
        
        

    return final_train_set, final_holdout_set

def accuracy(d,test_data): #checks for inaccuracy

    count=0
    incorrect=[]
    correct=[]
    old_data=test_data.index

    test_data=test_data.reset_index(drop=True)
    y_pred=[]

    for i in range(test_data.shape[0]):
        y_pred.append(d.predict(test_data.loc[i]))


    for i in range(len(y_pred)):
        if y_pred[i]== d.d1[test_data['Output'][i]]:
            count+=1
            correct.append(i)
        else:
            incorrect.append(i)
    
    new_data=[]
    for i in incorrect:
        new_data.append(old_data[i])  
    return count/len(test_data),new_data

print("Accuracy of 1st DTree:",accuracy(d,test_data)[0]*100,"%")
Training_Set, Holdout_Set = bagging_susbstition(train_data, test_data,d)

Accuracy of 1st DTree: 31.818181818181817 %


# Converting Indexes to Dataframe Function
<ol>
    <li> FUNCTIONS
        <ul>
            <li>convert_indices_to_DataFrame - Converts DataFrame from Numpy Array
                <ul>
                    <li>Parameters - 
                        <ul>
                            <li>Training_set - previous training set</li>
                            <li>d - Tree Object</li>
                        </ul>
                    </li>
                    <li>Returns - 
                        <ul>
                            <li>v1 - Array of indices</li>
                        </ul>
                    </li>
                </ul>
            </li>
        </ul>
    </li>
</ol>

In [10]:
def convert_indices_to_DataFrame(Training_Set,d): #converts dataframe from numpy array
    index1=[]
    Training_Set  
    d1=[]
    for i, j in d.dataframe().iterrows():
        if i in Training_Set:
            c1=Training_Set.count(i)
            for _ in range(c1):
                d1.append(d.dataframe()[i:i+1].values)
    v1=[]
    for i in d1:
        b1=[]
        for t in i:
            for r in t:
                b1.append(r)
        v1.append(b1)
    return v1


d1=DecisionTree()
Training_Set_d2 = pd.DataFrame(data= convert_indices_to_DataFrame(Training_Set,d1),columns=d1.dataframe().columns)
HoldOut_Set_d2 = pd.DataFrame(data=  convert_indices_to_DataFrame(Holdout_Set,d1),columns=d1.dataframe().columns)
d1.train(Training_Set_d2)

i =  White King file (a)
Information gain: -inf
i =  White King file (b)
Average entropy: 2.6864455666060127
Information gain: 0.129450351894699
i =  White King file (c)
Average entropy: 2.2938922859674267
Information gain: 0.5220036325332851
i =  White King file (d)
Average entropy: 2.31678348408587
Information gain: 0.49911243441484165
i =  White King rank (e)
Information gain: -inf
i =  White King file (f)
Information gain: -inf
i =  White King rank (g)
Information gain: -inf
i =  White King file (h)
Information gain: -inf
i =  White King rank (1)
Average entropy: 2.6751353050779643
Information gain: 0.14076061342274748
i =  White King rank (2)
Average entropy: 2.676615183653939
Information gain: 0.1392807348467726
i =  White King rank (3)
Average entropy: 2.702822855537323
Information gain: 0.1130730629633887
i =  White King rank (4)
Average entropy: 2.717318863490284
Information gain: 0.0985770550104279
i =  White King rank (5)
Information gain: -inf
i =  White King rank (6)
Infor

Average entropy: 2.3084724089939077
Information gain: 0.06172861975003041
i =  Black King rank (8)
Information gain: -inf
Splitting Tree  White Rook file (b)  with info gain  0.40928330430409265
i =  White King file (a)
Information gain: -inf
i =  White King file (b)
Average entropy: 1.7764605930931316
Information gain: 0.21447985947595716
i =  White King file (c)
Information gain: -inf
i =  White King file (d)
Average entropy: 1.7764605930931316
Information gain: 0.21447985947595716
i =  White King rank (e)
Information gain: -inf
i =  White King file (f)
Information gain: -inf
i =  White King rank (g)
Information gain: -inf
i =  White King file (h)
Information gain: -inf
i =  White King rank (1)
Average entropy: 1.7751593601716817
Information gain: 0.21578109239740706
i =  White King rank (2)
Average entropy: 1.7109864690920356
Information gain: 0.27995398347705325
i =  White King rank (3)
Average entropy: 1.7897739223862799
Information gain: 0.20116653018280894
i =  White King rank (

i =  White King file (a)
Information gain: -inf
i =  White King file (b)
Average entropy: 0.3185930346212532
Information gain: 0.07265052900800245
i =  White King file (c)
Information gain: -inf
i =  White King file (d)
Average entropy: 0.3185930346212532
Information gain: 0.07265052900800245
i =  White King rank (e)
Information gain: -inf
i =  White King file (f)
Information gain: -inf
i =  White King rank (g)
Information gain: -inf
i =  White King file (h)
Information gain: -inf
i =  White King rank (1)
Average entropy: 0.34840961638313933
Information gain: 0.04283394724611633
i =  White King rank (2)
Information gain: -inf
i =  White King rank (3)
Average entropy: 0.37188206585666517
Information gain: 0.019361497772590497
i =  White King rank (4)
Average entropy: 0.37188206585666517
Information gain: 0.019361497772590497
i =  White King rank (5)
Information gain: -inf
i =  White King rank (6)
Information gain: -inf
i =  White King rank (7)
Information gain: -inf
i =  White King rank

Information gain: -inf
i =  Black King rank (5)
Information gain: -inf
i =  Black King rank (6)
Information gain: -inf
i =  Black King rank (7)
Information gain: -inf
i =  Black King rank (8)
Information gain: -inf
Splitting Tree  White King file (a)  with info gain  -inf
i =  White King file (a)
Information gain: -inf
i =  White King file (b)
Information gain: -inf
i =  White King file (c)
Information gain: -inf
i =  White King file (d)
Information gain: -inf
i =  White King rank (e)
Information gain: -inf
i =  White King file (f)
Information gain: -inf
i =  White King rank (g)
Information gain: -inf
i =  White King file (h)
Information gain: -inf
i =  White King rank (1)
Information gain: -inf
i =  White King rank (2)
Information gain: -inf
i =  White King rank (3)
Information gain: -inf
i =  White King rank (4)
Information gain: -inf
i =  White King rank (5)
Information gain: -inf
i =  White King rank (6)
Information gain: -inf
i =  White King rank (7)
Information gain: -inf
i =  Wh

i =  White King file (a)
Information gain: -inf
i =  White King file (b)
Information gain: -inf
i =  White King file (c)
Information gain: -inf
i =  White King file (d)
Information gain: -inf
i =  White King rank (e)
Information gain: -inf
i =  White King file (f)
Information gain: -inf
i =  White King rank (g)
Information gain: -inf
i =  White King file (h)
Information gain: -inf
i =  White King rank (1)
Average entropy: 1.8128353980320053
Information gain: 0.020300197014401222
i =  White King rank (2)
Average entropy: 1.3853914838028087
Information gain: 0.4477441112435978
i =  White King rank (3)
Average entropy: 1.4588350430712191
Information gain: 0.3743005519751874
i =  White King rank (4)
Average entropy: 1.4728528900970448
Information gain: 0.36028270494936177
i =  White King rank (5)
Information gain: -inf
i =  White King rank (6)
Information gain: -inf
i =  White King rank (7)
Information gain: -inf
i =  White King rank (8)
Information gain: -inf
i =  White Rook file (a)
Aver

...

Information gain: -inf
i =  White Rook file (c)
Information gain: -inf
i =  White Rook file (d)
Average entropy: 0.9658770904834564
Information gain: 0.30189312562096404
i =  White Rook file (e)
Average entropy: 1.1974809498138532
Information gain: 0.0702892662905672
i =  White Rook file (f)
Average entropy: 1.2334790178943744
Information gain: 0.034291198210046
i =  White Rook file (g)
Average entropy: 1.1772954803560114
Information gain: 0.09047473574840903
i =  White Rook file (h)
Average entropy: 0.8540430168836101
Information gain: 0.41372719922081036
i =  White Rook rank (1)
Average entropy: 1.1772954803560114
Information gain: 0.09047473574840903
i =  White Rook rank (2)
Average entropy: 1.1772954803560114
Information gain: 0.09047473574840903
i =  White Rook rank (3)
Average entropy: 0.8140538769174757
Information gain: 0.45371633918694476
i =  White Rook rank (4)
Average entropy: 0.9658770904834564
Information gain: 0.30189312562096404
i =  White Rook rank (5)
Average entropy:

Information gain: -inf
i =  White King rank (6)
Information gain: -inf
i =  White King rank (7)
Information gain: -inf
i =  White King rank (8)
Information gain: -inf
i =  White Rook file (a)
Information gain: -inf
i =  White Rook file (b)
Information gain: -inf
i =  White Rook file (c)
Information gain: -inf
i =  White Rook file (d)
Information gain: -inf
i =  White Rook file (e)
Information gain: -inf
i =  White Rook file (f)
Information gain: -inf
i =  White Rook file (g)
Information gain: -inf
i =  White Rook file (h)
Information gain: -inf
i =  White Rook rank (1)
Information gain: -inf
i =  White Rook rank (2)
Information gain: -inf
i =  White Rook rank (3)
Information gain: -inf
i =  White Rook rank (4)
Information gain: -inf
i =  White Rook rank (5)
Information gain: -inf
i =  White Rook rank (6)
Information gain: -inf
i =  White Rook rank (7)
Information gain: -inf
i =  White Rook rank (8)
Information gain: -inf
i =  Black King file (a)
Information gain: -inf
i =  Black King f

Information gain: -inf
i =  Black King rank (5)
Information gain: -inf
i =  Black King rank (6)
Information gain: -inf
i =  Black King rank (7)
Information gain: -inf
i =  Black King rank (8)
Information gain: -inf
Splitting Tree  White King file (a)  with info gain  -inf
i =  White King file (a)
Information gain: -inf
i =  White King file (b)
Information gain: -inf
i =  White King file (c)
Information gain: -inf
i =  White King file (d)
Information gain: -inf
i =  White King rank (e)
Information gain: -inf
i =  White King file (f)
Information gain: -inf
i =  White King rank (g)
Information gain: -inf
i =  White King file (h)
Information gain: -inf
i =  White King rank (1)
Average entropy: 0.75
Information gain: 0.20443400292496505
i =  White King rank (2)
Average entropy: 0.6068441215341679
Information gain: 0.34758988139079716
i =  White King rank (3)
Average entropy: 0.0
Information gain: 0.954434002924965
i =  White King rank (4)
Information gain: -inf
i =  White King rank (5)
Info

Information gain: -inf
i =  White King file (c)
Information gain: -inf
i =  White King file (d)
Information gain: -inf
i =  White King rank (e)
Information gain: -inf
i =  White King file (f)
Information gain: -inf
i =  White King rank (g)
Information gain: -inf
i =  White King file (h)
Information gain: -inf
i =  White King rank (1)
Information gain: -inf
i =  White King rank (2)
Information gain: -inf
i =  White King rank (3)
Information gain: -inf
i =  White King rank (4)
Information gain: -inf
i =  White King rank (5)
Information gain: -inf
i =  White King rank (6)
Information gain: -inf
i =  White King rank (7)
Information gain: -inf
i =  White King rank (8)
Information gain: -inf
i =  White Rook file (a)
Information gain: -inf
i =  White Rook file (b)
Information gain: -inf
i =  White Rook file (c)
Information gain: -inf
i =  White Rook file (d)
Information gain: -inf
i =  White Rook file (e)
Information gain: -inf
i =  White Rook file (f)
Information gain: -inf
i =  White Rook f

Information gain: -inf
i =  Black King file (e)
Information gain: -inf
i =  Black King file (f)
Information gain: -inf
i =  Black King file (g)
Information gain: -inf
i =  Black King file (h)
Information gain: -inf
i =  Black King rank (1)
Information gain: -inf
i =  Black King rank (2)
Information gain: -inf
i =  Black King rank (3)
Information gain: -inf
i =  Black King rank (4)
Information gain: -inf
i =  Black King rank (5)
Information gain: -inf
i =  Black King rank (6)
Information gain: -inf
i =  Black King rank (7)
Information gain: -inf
i =  Black King rank (8)
Information gain: -inf
Splitting Tree  White King file (a)  with info gain  -inf
i =  White King file (a)
Information gain: -inf
i =  White King file (b)
Information gain: -inf
i =  White King file (c)
Information gain: -inf
i =  White King file (d)
Information gain: -inf
i =  White King rank (e)
Information gain: -inf
i =  White King file (f)
Information gain: -inf
i =  White King rank (g)
Information gain: -inf
i =  Wh

### Training set for decision tree #2

In [11]:
Training_Set_d2.head()

,White King file (a),White King file (b),White King file (c),White King file (d),White King rank (e),White King file (f),White King rank (g),White King file (h),White King rank (1),White King rank (2),...,Black King file (h),Black King rank (1),Black King rank (2),Black King rank (3),Black King rank (4),Black King rank (5),Black King rank (6),Black King rank (7),Black King rank (8),Output
0,0,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,17
1,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,17
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,17
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,17
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,17


### Validate set for decision tree #2

In [12]:
validate_data.head()

,White King file (a),White King file (b),White King file (c),White King file (d),White King rank (e),White King file (f),White King rank (g),White King file (h),White King rank (1),White King rank (2),...,Black King file (h),Black King rank (1),Black King rank (2),Black King rank (3),Black King rank (4),Black King rank (5),Black King rank (6),Black King rank (7),Black King rank (8),Output
170,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,8
64,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2
44,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
143,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,6
40,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1


### HoldOut set for decision tree #2

In [17]:
HoldOut_Set_d2.head()

,White King file (a),White King file (b),White King file (c),White King file (d),White King rank (e),White King file (f),White King rank (g),White King file (h),White King rank (1),White King rank (2),...,Black King file (h),Black King rank (1),Black King rank (2),Black King rank (3),Black King rank (4),Black King rank (5),Black King rank (6),Black King rank (7),Black King rank (8),Output
0,0,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,17
1,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,17
2,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,17
3,0,0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,17
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,17


### Accuracy of Decision Tree #2

In [18]:
print("Accuracy of 2nd DTree:",accuracy(d,HoldOut_Set_d2 )[0]*100,"%")

Accuracy of 2nd DTree: 61.36363636363637 %


In [19]:
def ensemble_tree(tree1,tree2,validation_set):
    
    
    accuracy_tree1 = accuracy(tree1,validation_set)[0]*100
    accuracy_tree2 = accuracy(tree2,validation_set)[0]*100
    
    if accuracy_tree1 > accuracy_tree2:
        return 'Tree1',accuracy_tree1
    else:
        return 'Tree2',accuracy_tree2
    
winning_tree,  validating_accuracy= ensemble_tree(d,d1,validate_data)
print('The winning tree is:',winning_tree,"with accuracy:",validating_accuracy,"%")

The winning tree is: Tree2 with accuracy: 38.63636363636363 %
